In [1]:
# dependencies
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import create_engine, func
from sqlalchemy import create_engine, inspect

In [2]:
phytoplankton = pd.read_csv("data/phytoplankton.csv")
zooplankton =  pd.read_csv("data/zooplankton.csv")
#phytoplankton_color = pd.read_csv("data/phytoplankton_ci.csv")

In [3]:

phytoplankton.head()

,FID,id,Route,Latitude,Longitude,SampleDateUTC,Year,Month,Day,Time_24hr,...,caab_code,taxon_start_date,taxon_per_m3,phyto_comments,acknowledgements,first_occurrence,parent_taxon_name,training,comments,geom
0,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880720,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,50566000.0,NaN,3333.3,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
1,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880722,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,52030054.0,NaN,666.7,*Noted as present in the zoop count*,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
2,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880721,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,52030900.0,2011-05-17,666.7,*Noted as present in the zoop count*,Wallenius Wilhelmsen,2011-09-20,Tripos spp.,"Internal training Pru Bonhma, Gustaaf Hallegraeff",NaN,POINT (135.33099 -37.91378)
3,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880726,ABAB,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,99530018.0,2011-05-17,6666.7,NaN,Wallenius Wilhelmsen,2011-09-15,Pseudo-nitzschia spp.,"Internal training Pru Bonhma, Gustaaf Hallegraeff",NaN,POINT (134.91247 -37.87656)
4,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880727,ABAB,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,NaN,NaN,3333.3,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (134.91247 -37.87656)


In [ ]:
#rounding coordinates
phytoplankton = phytoplankton.round(2)
phytoplankton.head()

In [ ]:
#putting lat and long into one column

#phytoplankton["coordinates"] = "[" + phytoplankton["Latitude"].astype(str) +", " + phytoplankton["Longitude"].astype(str) + "]"

In [4]:
#drop unneeded columns for phytoplankton APIs
phyto = phytoplankton.drop(["id",
                                    "Time_24hr",
                                    "FID",
                                    "SampleDateUTC",
#                                    "Latitude",
#                                    "Longitude",
                                    "Month",
                                    "Day",
                                    "family",
                                    "genus",
#                                    "taxon_group",
                                    "species",
                                    "caab_code",
                                   "taxon_start_date",
                                    "taxon_name",
                                   "phyto_comments",
                                   "acknowledgements",
                                   "first_occurrence",
                                   "parent_taxon_name",
                                   "training",
                                   "comments",
                                    "geom"
                                  ],
                                  axis=1)

In [5]:
phyto.head()

,Route,Latitude,Longitude,Year,taxon_group,taxon_per_m3
0,ABAB,-37.9138,135.3310,2020,Ciliate,3333.3
1,ABAB,-37.9138,135.3310,2020,Dinoflagellate,666.7
2,ABAB,-37.9138,135.3310,2020,Dinoflagellate,666.7
3,ABAB,-37.8766,134.9125,2020,Pennate diatom,6666.7
4,ABAB,-37.8766,134.9125,2020,Dinoflagellate,3333.3


# phyto_taxon_group API

In [6]:
phyto_taxon_group = phyto.drop(["Latitude", "Longitude"], axis=1)

phyto_taxon_group.head()

,Route,Year,taxon_group,taxon_per_m3
0,ABAB,2020,Ciliate,3333.3
1,ABAB,2020,Dinoflagellate,666.7
2,ABAB,2020,Dinoflagellate,666.7
3,ABAB,2020,Pennate diatom,6666.7
4,ABAB,2020,Dinoflagellate,3333.3


# phyto_groupby_taxon


In [7]:
phyto_groupby_taxon = pd.DataFrame(phyto_taxon_group.groupby(by=["taxon_group","Year"])["taxon_per_m3"].mean())

phyto_groupby_taxon.head()

taxon_per_m3
taxon_group    Year              
Centric diatom 2007   7551.534278
               2008   6175.653846
               2009   8697.562267
               2010  11326.249533
               2011   6213.341902

# phyto_routes API

In [8]:
# Routes API
# reset index and and a log of taxon_main_routes for more informative heatmap
phyto_routes = phyto.drop(["taxon_group"], axis=1)

phyto_routes = phyto_routes.reset_index().drop(["index"],axis=1)
phyto_routes["log2_m3"] = np.log2(phyto_routes["taxon_per_m3"]).round(1)
phyto_routes.head()

,Route,Latitude,Longitude,Year,taxon_per_m3,log2_m3
0,ABAB,-37.9138,135.3310,2020,3333.3,11.7
1,ABAB,-37.9138,135.3310,2020,666.7,9.4
2,ABAB,-37.9138,135.3310,2020,666.7,9.4
3,ABAB,-37.8766,134.9125,2020,6666.7,12.7
4,ABAB,-37.8766,134.9125,2020,3333.3,11.7


# phyto_groupby_route

In [9]:
# find the average taxon per m3 grouped by each route and year and turn it into a dataframe
phyto_groupby_route = pd.DataFrame(phyto_routes.groupby(by=["Route","Year"])["taxon_per_m3"].mean())

phyto_groupby_route.head()

taxon_per_m3
Route Year              
ABAB  2020   3414.812222
ABFR  2014   4933.332667
      2015   4237.531034
      2016   5445.455455
      2017   2715.704412

In [ ]:
# get list of unique routes to unimportant ones for per year sample consistency/data limit
column_values = phytoplankton[["Route"]].values.ravel()
unique_routes =  pd.unique(column_values)

print(unique_routes)

In [ ]:
#total count of unique values

item_counts = phytoplankton["Route"].value_counts()
print(item_counts)

In [ ]:
#for loop to check which routes to drop

for route in unique_routes:
    print(route)
    route_filter = phytoplankton.loc[phytoplankton['Route'] == route]
    print(route_filter["Year"].value_counts())
    print("***************************************************")

In [10]:
#have only four main routes for sample size due to data limits

phyto_main_4 = phyto_routes.loc[((phyto_routes['Route'] == "ANHO") |
        (phyto_routes["Route"] == 'BRSY')|
        (phyto_routes["Route"] == 'HOAN')|
#        (phyto_routes["Route"] == 'PKME')|
        (phyto_routes["Route"] == 'MEAD')), :]

# phyto_years API 

### Use when  x = Year NOT Route

In [11]:
#phyto_main_4 = phyto_main_routes.drop(["index"],axis=1
phyto_years = phyto_main_4.reset_index().drop(["index", "log2_m3"],axis=1)
phyto_years.head()

,Route,Latitude,Longitude,Year,taxon_per_m3
0,ANHO,-60.0961,89.6107,2007,666.7
1,ANHO,-63.2677,139.8854,2008,3333.3
2,ANHO,-63.2677,139.8854,2008,10000.0
3,ANHO,-63.2677,139.8854,2008,3333.3
4,ANHO,-63.2677,139.8854,2008,6666.7


In [ ]:
phyto_routes.count


# Zooplankton

In [ ]:
#rounding coordinates
zooplankton = zooplankton.round(2)
zooplankton.head()

In [ ]:
#coordinates column
#zooplankton["coordinates"] = "[" + zooplankton["Latitude"].astype(str) +", " + zooplankton["Longitude"].astype(str) + "]"

In [ ]:
for col in zooplankton.columns:
    print(col)

In [ ]:
#drop unneeded columns for zooplankton
zoo = zooplankton.drop(["id",
                                "FID",
                                "Time_24hr",
                                "SampleDateUTC",
#                                "Latitude",
#                                "Longitude",
                                "family",
                                "Month",
                                "Day",
                                "genus",
#                                "taxon_group",
                                "species",
                                "sex",
                                  "life_stage",
                                    "caab_code",
                                    "aphiaid",  
                                   "taxon_start_date",
                                   "zoop_comments",
                                   "acknowledgements",
                                   "first_occurrence",
                             "taxon_name",
                                   "parent_taxon_name",
                                   "training",
                                   "comments",
                                    "taxon_eco_group",
                                "geom"
                                  ],
                                  axis=1)

In [ ]:
zoo.head()

# zoo_taxon_group API

In [ ]:
zoo_taxon_group = zoo.drop(["Latitude", "Longitude"], axis=1)
zoo_taxon_group.head()

# zoo_groupby_taxon

In [ ]:
zoo_groupby_taxon = pd.DataFrame(zoo_taxon_group.groupby(by=["taxon_group","Year"])["taxon_per_m3"].mean())
zoo_groupby_taxon.head()

# zoo_routes API

In [ ]:
#reset add column for logarithmic scale
zoo_routes = zoo.drop(["taxon_group"], axis =1)
zoo_routes = zoo_routes.reset_index().drop(["index"],axis=1)
zoo_routes["log2_m3"] = np.log2(zoo_routes["taxon_per_m3"]).round(1)
zoo_routes.head()

# zoo_groupby_route API

In [ ]:
# find the average taxon per m3 grouped by each route and year and turn it into a dataframe
zoo_groupby_route = pd.DataFrame(zoo_routes.groupby(by=["Route","Year"])["taxon_per_m3"].mean())

zoo_groupby_route.head()

In [ ]:
zooplankton.count

In [ ]:
column_values = zooplankton[["Route"]].values.ravel()
unique_routes =  pd.unique(column_values)

print(unique_routes)

In [ ]:
#for loop to check which routes to drop

for route in unique_routes:
    print(route)
    route_filter = zooplankton.loc[zooplankton['Route'] == route]
    print(route_filter["Year"].value_counts())
    print("***************************************************")

In [ ]:
#ANHO, BRSY, HOAN, MEAD, PKME

In [ ]:
zoo_main_4 = zoo_routes.loc[((zoo_routes['Route'] == "ANHO") |
        (zoo_routes["Route"] == 'BRSY')|
        (zoo_routes["Route"] == 'HOAN')|
#        (zoo_routes["Route"] == 'PKME')|
        (zoo_routes["Route"] == 'MEAD')), :]

# zoo_years API

In [ ]:
zoo_years = zoo_main_4.reset_index().drop(["index", "log2_m3"], axis=1)
zoo_years.head()

In [ ]:
print(np.log2(zoo_routes["taxon_per_m3"].max()).round(1))


In [ ]:
print(np.log2(phyto_routes["taxon_per_m3"].max()).round(1))

In [ ]:
# # save to csv, remember to delete index in csv to save space
# zoo_main_routes.to_csv("clean_data/zooplankton_clean.csv")
# phyto_main_routes.to_csv("clean_data/phytoplankton_clean.csv")

In [ ]:
# for col in phytoplankton_color.columns:
#     print(col)

In [ ]:
# #drop unneeded columns for phytoplankton color index
# phytoplankton_color = phytoplankton_color.drop(["FID",
#                                                 "start_port",
#                                                 "end_port",
#                                                 "trip_code",
#                                                 "colour_index",
#                                                 "measurement",
#                                                "acknowledgements",
#                                                "route_frequency",
#                                                "route_start_date",
#                                                "pci_comments",
#                                                 "geom",
#                                                   ],
#                                                   axis=1)

In [ ]:
phytoplankton_color

In [ ]:
# add csv to data base function
# def read_sql(data):
#     data.to_sql(f"'{data}'", conn, if_exists = "append")


In [ ]:
# # add csv to data base
# read_sql(phytoplankton)
# read_sql(zooplankton)
# read_sql(phytoplankton_color)

In [ ]:
# Create an engine for the plankton.sqlite database
engine = create_engine("sqlite:///planktontestmain4.sqlite", echo=False)
conn = engine.connect()

# phyto_routes
# phyto_taxon_group
# phyto_years
# phyto_groupby_route

# zoo_routes
# zoo_taxon_group
# zoo_years
# zoo_groupby_route

In [ ]:
phyto_routes.head()

In [ ]:
phyto_routes.to_sql("phytoplankton", conn,)
phyto_taxon_group.to_sql("phytoplankton_taxon_group", conn,)
phyto_years.to_sql("phytoplankton_years", conn,)
zoo_routes.to_sql("zooplankton", conn,)
zoo_taxon_group.to_sql("zooplankton_taxon_group", conn,)
zoo_years.to_sql("zooplankton_years", conn,)
zoo_groupby_route.to_sql("zooplankton_groupby_route", conn,)
phyto_groupby_route.to_sql("phytoplankton_groupby_route", conn,)
#phytoplankton_color.to_sql("phytoplankton_color_index", conn,)

In [ ]:
zoo_groupby_taxon.to_sql("zooplankton_groupby_taxon", conn,)
phyto_groupby_taxon.to_sql("phytoplankton_groupby_taxon", conn,)

In [ ]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

In [ ]:
# Use `engine.execute` to select and display the first 10 rows from the station table
engine.execute('SELECT * FROM zooplankton LIMIT 5').fetchall()

In [ ]:
# Query phytoplankton Records in the the Database
data = pd.read_sql("SELECT * FROM phytoplankton", conn)
dataZ = pd.read_sql("SELECT * FROM phytoplankton_groupby_taxon", conn)

In [ ]:
data.head()

In [ ]:
dataZ.head()

In [ ]:
zoo_years = pd.read_sql("SELECT * FROM phytoplankton_years WHERE Year > 2016", conn)
zoo_years.head()